In [2]:
import joblib
from skimage.measure import label, regionprops
from ultralytics import YOLO
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import csv

C:\Users\KIIT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


C:\Users\KIIT\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
def Length_predictor(length, ratio):
    Length_model = joblib.load("Length_predictor.joblib")
    Length_cm = Length_model.predict([[length, ratio]])[0]
    return Length_cm

In [4]:
def Width_predictor(width):
    Width_model = joblib.load("Width_predictor.joblib")
    Width_cm = Width_model.predict([[width]])[0]
    return Width_cm

In [5]:
def Volume_predictor(length, breadth):
    Volume_model = joblib.load("Volume_predictor_1.joblib")
    Volume = Volume_model.predict([[length, breadth]])[0]
    return Volume

In [6]:
def Shell_Weight_predictor(Volume):
    Shell_Weight_model = joblib.load("Shell_Weight_predictor_1.joblib")
    Shell_Weight = Shell_Weight_model.predict([[Volume]])[0]
    return Shell_Weight

In [7]:
def Grading(Shell_Weight):
    Grading_model = joblib.load("Grading.joblib")
    Grade = Grading_model.predict([[Shell_Weight]])[0]
    return Grade

In [8]:
def find_contours_from_yolo(img_path, model_path):
    # Load the YOLOv8 model
    model = YOLO(model_path)

    # Load and preprocess the input image
    img = cv2.imread(img_path)

    # Check if the image is loaded correctly
    if img is None:
        raise ValueError("Image not found or the path is incorrect.")
    
    print(f"Image shape: {img.shape}")

    # Check if the image is grayscale or color
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
    H, W, _ = img.shape

    # Inference with the YOLOv8 model
    results = model(img)

    contours_list = []
    bounding_rects = []

    # Process each detection in the results
    for result in results:
        for j, mask in enumerate(result.masks.data):
            mask = mask.cpu().numpy().astype(np.uint8) * 255  # Convert to 8-bit binary mask

            # Check if mask is empty
            if mask.size == 0:
                raise ValueError("Mask is empty. Please check the model's output.")
            
            # print(f"Mask shape before resize: {mask.shape}")

            # Resize mask to the original image size
            mask = cv2.resize(mask, (W, H))

            # print(f"Mask shape after resize: {mask.shape}")

            # Find the contours of the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not contours:
                raise ValueError("No contours found in the mask. Please check the mask and try again.")
            
            contours_list.append(contours)
            
            # Compute bounding rectangles for each contour
            for contour in contours:
                x, y, width, height = cv2.boundingRect(contour)
                bounding_rects.append((x, y, width, height))
                
                # # Optional: Draw the contour and bounding rectangle on the original image for visualization
                # cv2.drawContours(img, [contour], -1, (0, 255, 0), 2)
                # cv2.rectangle(img, (x, y), (x+width, y+height), (255, 0, 0), 2)
                # cv2.imshow('Mask Contours and Bounding Rectangles', img)
                # cv2.waitKey(0)
                # cv2.destroyAllWindows()
            
    return contours_list, bounding_rects

In [9]:
def predict_cocoon_1(bounding_rects):
    
    for i, rect in enumerate(bounding_rects):
        x, y, w, h = rect
        # print(f"Bounding rectangle {i}: x={x}, y={y}, width={w}, height={h}")
        print(w, h)
        length_cm = w * 0.0264583333  # Example conversion factor
        breadth_cm = h * 0.0264583333  # Example conversion factor
        ratio = 4.19460691
        length = Length_predictor(length_cm, ratio)
        width = Width_predictor(breadth_cm)
        volume = Volume_predictor(length, width)
        shell_weight = Shell_Weight_predictor(volume)
        # grade = Grading(shell_weight)
        # print(f"Length: {length_cm} cm, Breadth: {breadth_cm} cm, Volume: {volume} cm^3, Shell Weight: {shell_weight} g, Grade: {grade}")
        return length_cm ,breadth_cm, length ,width , volume, shell_weight
    

In [11]:
image_path = input("Enter image path: ")
model_path = '/Users/KIIT/Desktop/Research/CTR&TI/Models/runs/segment/train32/weights/last.pt'  # Path to your trained YOLOv8 model

contours_list, bounding_rects = find_contours_from_yolo(image_path, model_path)

# for i, contours in enumerate(contours_list):
#     print(f"Contours for mask {i}: {contours}")

length_cm, breadth_cm, length, width, volume, shell_weight = predict_cocoon_1(bounding_rects)
print(f"Predicted Length: {length_cm} cm, Predicted_Breadth: {breadth_cm} cm, Real length: {length}, Real width: {width}, Volume: {volume} cm^3, Shell Weight: {shell_weight} g")
# csv.data[length, breadth, volume, shell_weight, grade]

Enter image path:  /Users/KIIT/Desktop/Research/CTR&TI/Paper/ppt/DSC04385.JPG


Image shape: (2813, 2053, 3)

0: 640x480 1 Cocoon, 554.2ms
Speed: 9.7ms preprocess, 554.2ms inference, 16.0ms postprocess per image at shape (1, 3, 640, 480)
902 1490
Predicted Length: 23.865416636600003 cm, Predicted_Breadth: 39.422916617 cm, Real length: 5.686475479793893, Real width: 2.9175010684183, Volume: 30.809532802113957 cm^3, Shell Weight: 1.6285789048455188 g


In [10]:
def predict_cocoon(img):
    model_path = '/Users/KIIT/Desktop/Research/CTR&TI/Models/runs/segment/train32/weights/last.pt'

    img = cv2.imread(image_path)
    if img is None:
          print("Error: Could not read image!")
  
    print(img.shape)
    
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
    H, W, _ = img.shape
    print(H)
    print(W)
    # image = load_img(img_path, target_size=(480, 640))  # Change target size to match your model's input size
    img = img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0) 

    model = YOLO(model_path)

    results = model(img)

    for result in results:
        for j, mask in enumerate(result.masks.data):

            mask = mask.cpu().numpy().astype(np.uint8) * 255
            if mask.size == 0:
                    raise ValueError("Mask is empty. Please check the model's output.")

            mask = cv2.resize(mask, (H, W))

            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
            if not contours:
                raise ValueError("No contours found in the mask. Please check the mask and try again.")

            cocoon_contour = max(contours, key=cv2.contourArea)
            # cv2.imwrite('./output.png', mask)
            x, y, width, height = cv2.boundingRect(cocoon_contour)

            # Convert pixel measurements to centimeters (assuming you know the conversion factor)
            cm_per_pixel = 0.0264583333  # Example value, you need to calibrate this
            length_cm = width*cm_per_pixel
            breadth_cm = height*cm_per_pixel

            # print(f"Length: {length_cm} cm, Breadth: {breadth_cm} cm")

            Volume = Volume_predictor(length_cm, breadth_cm)

            Shell_Weight = Shell_Weight_predictor(Volume)

            Grade = Grading(Shell_Weight)

            return length_cm, breadth_cm, Volume, Shell_Weight, Grade


In [13]:

import pandas as pd

csv_file_path = "Comparision.csv"

model_path = '/Users/KIIT/Desktop/Research/CTR&TI/Models/runs/segment/train32/weights/last.pt'  # Path to your trained YOLOv8 model

folder_path = input("Enter folder path:")

Original_file_path = 'dataset/The_final_dataset_1.csv'

creating_csv(model_path, csv_file_path, folder_path, Original_file_path)

KeyboardInterrupt: Interrupted by user

In [60]:
def creating_csv(model_path, csv_file_path, folder_path, Original_file_path):
    
    headers = ['S.No.','filename','Length (cm)', 'Original Length', 'Width (cm)', 'Original Width', 'Volume (cm^3)', 'Original Volume', 'Shell Weight (g)', 'Original Shell Weight']
    csv_data = []
    
    file = pd.read_csv(Original_file_path)
    
    Original_Length = file['Cocoon length(cm)']
    Original_Width = file['mean width(cm)']
    Original_Volume = file['Cocoon Volume (cm3)']
    Original_Shell_Weight = file['Shell Weight']
    
    Original_Length = Original_Length[103: 441].tolist()
    Original_Width = Original_Width[103: 441].tolist()
    Original_Volume = Original_Volume[103: 441].tolist()
    Original_Shell_Weight = Original_Shell_Weight[103: 441].tolist()
    
    j=0
    i=0
    k=0
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".JPG") or filename.endswith(".png"):  # Add other image formats if needed
            # print('twill')
            image_path = os.path.join(folder_path, filename)
            contours_list, bounding_rects = find_contours_from_yolo(image_path, model_path)
            
            length_cm, breadth_cm, length, width, volume, shell_weight = predict_cocoon_1(bounding_rects)
            
            if(i%2!=0):
                csv_data.append([i, filename, length, j, width, j, volume, j, shell_weight, j])
                print(csv_data[i])
            else:
                csv_data.append([i, filename, length, Original_Length[k], width, Original_Width[k], volume, Original_Volume[k], shell_weight, Original_Shell_Weight[k]])
                print(csv_data[i])
                k=k+1
            print(i)
            i=i+1
            
#             try:
#                 contours_list, bounding_rects = find_contours_from_yolo(image_path, model_path)
#                 length, breadth, volume, shell_weight, grade = predict_cocoon_1(bounding_rects)
#                 print("Picture:",filename)
#                 print(f"Length: {length} cm, Breadth: {breadth} cm, Volume: {volume} cm^3, Shell Weight: {shell_weight} g, Grade: {grade}")
#                 csv_data.append([filename, length, breadth, volume, shell_weight, grade])
#                 # print(csv_data)
#             except Exception as e:
#                 print(f"Error processing image {filename}: {e}")

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(csv_data)
        
        
    print(f"Data written to {csv_file_path} successfully.")


In [43]:
def predict_cocoon_grade(image_path):
    # Load the pre-trained segmentation model (example: U-Net, Mask R-CNN)
    # Assuming the segmentation model is already trained and available
    model_path = "/Users/KIIT/Desktop/Research/CTR&TI/Models/runs/segment/train32/weights/last.pt"
    model = YOLO(model_path)
    # Read the image
    image = cv2.imread(image_path)

    # Perform segmentation to get the mask
    results = model(image)
    
    mask = None
    
    for result in results:
        if hasattr(result, 'mask'):
            mask = result.masks.data
            break
    
    if mask is None:
        print("Error: No masks found")
        return None
    
    # print(type(mask))
    # # Assuming the mask has multiple channels or a different data type
    # if len(mask.shape) > 2:
    #     mask = mask[:, :, 0]  # Select the first channel (assuming it represents the segmentation)
    # mask = mask.astype(np.uint8)

    # Find the contours of the cocoon
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cocoon_contour = max(contours, key=cv2.contourArea)

    # Draw the contour on the original image (optional for visualization)
    cv2.drawContours(image, [cocoon_contour], -1, (0, 255, 0), 2)

    # Calculate bounding box to get length and breadth
    x, y, width, height = cv2.boundingRect(cocoon_contour)

    # Convert pixel measurements to centimeters (assuming you know the conversion factor)
    cm_per_pixel = 0.0264583333  # Example value, you need to calibrate this
    length_cm = width*cm_per_pixel
    breadth_cm = height*cm_per_pixel

    # print(f"Length: {length_cm} cm, Breadth: {breadth_cm} cm")
    
    Volume = Volume_predictor(length_cm, breadth_cm)
    
    Shell_Weight = Shell_Weight_predictor(Volume)
    
    Grade = Grading(Shell_Weight)
    
    return length_cm, breadth_cm, Volume, Shell_Weight, Grade
    